##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# DeepDream

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/beta/tutorials/generative/deepdream"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/generative/deepdream.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/generative/deepdream.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/r2/tutorials/generative/deepdream.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

This tutorial contains a minimal implementation of DeepDream, as described in this [blog post](https://ai.googleblog.com/2015/06/inceptionism-going-deeper-into-neural.html) by Alexander Mordvintsev.

DeepDream is an experiment that visualizes the patterns learned by a neural network. Similar to when a child watches clouds and tries to interpret random shapes, DeepDream over-interprets and enhances the patterns it sees in an image.

It does so by forwarding an image through the network, then calculating the gradient of the image with respect to the activations of a particular layer. The image is then modified to increase these activations, enhancing the patterns seen by the network, and resulting in a dream-like image. This process was dubbed "Inceptionism" (a reference to [InceptionNet](https://arxiv.org/pdf/1409.4842.pdf), and the movie Inception).

Let's demonstrate how you can make a neural network "dream" and enhance the surreal patterns it sees in an image.

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x  # Colab only.
except Exception:
  pass
import tensorflow as tf

from IPython.display import clear_output
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.figsize'] = (8, 8)
mpl.rcParams['axes.grid'] = False

## Choose an image
For this tutorial, let's use the image of a [Green Sea Turtle](https://commons.wikimedia.org/wiki/File:Green_Sea_Turtle_grazing_seagrass.jpg).

In [0]:
url = 'https://storage.googleapis.com/download.tensorflow.org/example_images/Green_Sea_Turtle_grazing_seagrass.jpg'

In [0]:
# Download an image and read it into a NumPy array.
def download(url):
  name = url.split('/')[-1]
  image_path = tf.keras.utils.get_file(name, origin=url)
  img = tf.keras.preprocessing.image.load_img(image_path)
  return img

# Normalize an image
def deprocess(img):
  return (img + 1.0)/2.0

original_img = download(url)
plt.imshow(original_img)

## Prepare the feature extraction model

Let's load a pre-trained image classification model. For this task, a pre-trained [InceptionV3](https://keras.io/applications/#inceptionv3) is used, similar to the network originally used by the authors.

In [0]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet')

The idea in DeepDream is to choose a layer (or layers) and maximize the "loss" in a way that the image increasingly "excites" the layers. The complexity of the features incorporated depends on layers chosen by you, i.e, lower layers produce strokes or simple patterns, while deeper layers give sophisticated features in images, or even whole objects.

In [0]:
# Maximize the activations of these arbitarily chosen layers
names = ['mixed2', 'mixed3', 'mixed5']
layers = [base_model.get_layer(name).output for name in names]

# Create our feature extraction model
dream_model = tf.keras.Model(inputs=base_model.input, outputs=layers)

## Calculate loss

The loss is the sum of the activations in the chosen layers. The loss is normalizaed at each layer so the contribution from larger layers does not outweigh smaller layers. Normally, loss is a quantity you wish to minimize via gradient descent. In DeepDream, you will maximize this loss via gradient ascent.

In [0]:
def calc_loss(image):
  image = image[None, ...]
  layer_activations = dream_model(image)

  losses = []
  for act in layer_activations:
    loss = tf.math.reduce_mean(act)
    losses.append(loss)
  total_loss = tf.add_n(losses)
  return total_loss

## Gradient ascent

Once you have calculated the loss for the chosen layers, all that is left is to calculate the gradients with respect to the image, and add them to the original image. 

Adding the gradients to the image enhances the patterns seen by the network. At each step, you will have created an image that increasingly excites the activations of certain layers in the network.

In [0]:
img = tf.keras.preprocessing.image.img_to_array(original_img)
img = tf.Variable(tf.keras.applications.inception_v3.preprocess_input(img))

In [0]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.005)

In [0]:
@tf.function
def deepdream():
  with tf.GradientTape() as tape:
    loss = -calc_loss(img)
  gradient = tape.gradient(loss, img)

  optimizer.apply_gradients([(gradient, img)])
  img.assign(tf.clip_by_value(img, -1, 1))
  return loss

## Result
Brace yourself, things are going to get dreamy!

In [0]:
STEPS = 500

In [0]:
for step in range(STEPS):
  loss = deepdream()
  if step % 50 == 0:
    clear_output(wait=True)
    plt.imshow(deprocess(img))
    plt.show()
    print ('Step %d, loss %f' % (step, -1.0*loss))
clear_output()
plt.imshow(deprocess(img))

## Next steps

Now that you have seen how to make a network "dream", go ahead and experiment with a different combination of layers, a different [pre-trained model](https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/applications) or your own neural network. You may also modify this code to maximize the activations of certain filters within a layer, as opposed to the layer as a whole.

Finally, you may wish to see this complete [implementation](https://github.com/tensorflow/tensorflow/tree/master/tensorflow/examples/tutorials/deepdream) in an older version of TensorFlow which includes additional tricks.